In [ ]:
%%configure -f
{"defaultLakehouse": {"name": "lh_cfg"}}

In [ ]:
# PL: Please turn off your "AutoSave" (Edit --> AutoSave --> Off)

In [ ]:
# kill me

# Temp cell for emulating the parameters, passed by pl_master --> nb_master

is_debug         = False
do_test_function = False
called_from = "bronze" # (silver, gold, power_bi, etc.)

import delta.tables as dlt
import json as j
from types import SimpleNamespace as sn
from pyspark.sql import functions as sf
from pyspark.sql import types as st
from inspect import stack as stk
import time as t
import datetime as dt
import pytz as pt
import re
import unicodedata as ucd
import pandas as pd
import requests as req
if is_debug:
    from pyspark.sql import SparkSession as ss

global_parameter = """
{
    "days_to_keep_log": 365,
    "send_success_email": "True",
    "send_error_email": "True",
    "email_on_success": "example@example.com",
    "email_on_error": "example@example.com",
    "time_zone_nb": "Canada/Eastern",
    "time_zone_pl": "Eastern Standard Time",
    "mssql_isolation_level": "READ_UNCOMMITTED",
    "projet_documentation_data_file_name": "configuration",
    "abfs_path_lh_log": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc",
    "abfs_path_lh_bronze": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc",
    "abfs_path_lh_silver": "abfss://abc@onelake.dfs.fabric.microsoft.com/1abc",
    "abfs_path_lh_cfg": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc",
    "now_dt": \"2024-05-07 14:09:09.367",
    "extraction_timestamp": "20240507_140909367"
}
"""

In [ ]:
# kill me

# Create local variables from the parameters, paseed by "pl_master"
global_parameter = j.loads(global_parameter) # Convert string to mapping
global_parameter = sn(**global_parameter) # Create variables from mapping

if is_debug:
    print(f"----- Test global_parameter - Start -----")
    print(f"global_parameter.days_to_keep_log:                    {global_parameter.days_to_keep_log}")
    print(f"global_parameter.send_success_email:                  {global_parameter.send_success_email}")
    print(f"global_parameter.send_error_email:                    {global_parameter.send_error_email}")
    print(f"global_parameter.email_on_success:                    {global_parameter.email_on_success}")
    print(f"global_parameter.email_on_error:                      {global_parameter.email_on_error}")
    print(f"global_parameter.time_zone_nb:                        {global_parameter.time_zone_nb}")
    print(f"global_parameter.time_zone_pl:                        {global_parameter.time_zone_pl}")
    print(f"global_parameter.mssql_isolation_level:               {global_parameter.mssql_isolation_level}")
    print(f"global_parameter.projet_documentation_data_file_name: {global_parameter.projet_documentation_data_file_name}")
    print(f"global_parameter.abfs_path_lh_log:                    {global_parameter.abfs_path_lh_log}")
    print(f"global_parameter.abfs_path_lh_bronze:                 {global_parameter.abfs_path_lh_bronze}")
    print(f"global_parameter.abfs_path_lh_silver:                 {global_parameter.abfs_path_lh_silver}")
    print(f"global_parameter.abfs_path_lh_cfg:                    {global_parameter.abfs_path_lh_cfg}")
    print(f"global_parameter.now_dt:                              {global_parameter.now_dt}")
    print(f"global_parameter.extraction_timestamp:                {global_parameter.extraction_timestamp}")
    print(f"----- Test global_parameter - End -----")

In [ ]:
# Print current session info
if is_debug:
    print(f"----- Current session info - Start -----")
    display(ss.getActiveSession())
    print(f"----- Current session info - End -----")

In [ ]:
%run nb_system_function

In [ ]:
# kill me

# Overwrite global_parameters - now_dt and extraction_timestamp

now_dt               = fn_get_now("datetime")[1]
extraction_timestamp = fn_get_now("string")[1]

global_parameter.now_dt               = now_dt
global_parameter.extraction_timestamp = extraction_timestamp

print(global_parameter.now_dt)
print(global_parameter.extraction_timestamp)

# Dataframe

## Add metadata columns

In [ ]:
# From table "lh_cfg.metadata_column"
def fn_add_metadata_column(
    sdf,
    dict_parameter
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug:
        par = {}

    try:
        # PL: Add warnings where needed

        # Collect metadata columns
        sdf_metadata_column = fn_execute_spark_sql(f"""
            SELECT DISTINCT `name`
            FROM `metadata_column`
            WHERE
                `technology`    = '{dict_parameter["technology"]}'
                AND `frequency` = '{dict_parameter["frequency"]}';
        """)[1]

        list_metadata_column = []
        for row in sdf_metadata_column.collect():
            list_metadata_column.append(row.name)

        # Add metadatacolumn "fa_date_extracted"
        if "fa_date_extracted" in list_metadata_column:
            sdf = sdf.withColumn("fa_date_extracted", sf.lit(global_parameter.now_dt).cast("Timestamp"))

        # Add metadatacolumn "fa_server_name"
        if "fa_server_name" in list_metadata_column:
            sdf = sdf.withColumn("fa_server_name", sf.lit(dict_parameter["server_name"]))

        # Add metadatacolumn "fa_database_name"
        if "fa_database_name" in list_metadata_column:
            sdf = sdf.withColumn("fa_database_name", sf.lit(dict_parameter["database_name"]))

        # Add metadatacolumn "fa_schema_name"
        if "fa_schema_name" in list_metadata_column:
            sdf = sdf.withColumn("fa_schema_name", sf.lit(dict_parameter["schema_name"]))

        # Add metadatacolumn "fa_table_name"
        if "fa_table_name" in list_metadata_column:
            sdf = sdf.withColumn("fa_table_name", sf.lit(dict_parameter["table_name"]))

        sdf_count = sdf.count()

        if is_debug:
            par["locals"] = locals()
            par["sdf"] = sdf.show(truncate = False, n = 5)
            fn_print_debug_info("Success", fn_name, par)

        return (True, sdf, sdf_count)
    except Exception as ex:
        ex = str(ex)

        if is_debug:
            par["locals"] = locals()
            par["exception"] = ex
            fn_print_debug_info("Error", fn_name, par)

        sdf            = str(sdf)
        dict_parameter = j.dumps(dict_parameter)
        if is_debug:
            par = str(par)
        fn_lh_log_log_insert(global_parameter.extraction_timestamp, "Silver", fn_name, str(locals()), "Danger", ex)

        return (False, ex)

# Test
if is_debug == True and do_test_function == True:
    print(f"\n----- Test fn_add_metadata_column() - Start -----\n")
    sdf = spark.createDataFrame(
        [(1, "column_1", "String", "alias_col_1"),
         (2, "column_2", "Integer", "alias_col_2")],
        ["sequence", "column_name", "data_type", "alias_name"])
    dict_parameter = {
        "technology": "Excel",
        "frequency": "Daily",
        "server_name": "Reference",
        "database_name": "Data",
        "schema_name": "Rapport_sur les coûts de dist. des addendas",
        "table_name": "Rapport"
    }
    print(f"Input value (sdf) type:            {type(sdf)}")
    print(f"Input value (sdf):")
    sdf.show(truncate = False, n = 5)
    print(f"Input value (dict_parameter) type: {type(dict_parameter)}")
    print(f"Input value (dict_parameter):      {dict_parameter}")
    print(f"\n# Execution (try) - Start #")
    time_start = dt.datetime.now()
    rv_try = fn_add_metadata_column(sdf, dict_parameter)
    print(f"# Execution (try) - End (duration: {(dt.datetime.now() - time_start)}) #\n")
    print(f"Return value:               {rv_try}")
    print(f"Return value (status) type: {type(rv_try[0])}")
    print(f"Return value (status):      {rv_try[0]}")
    print(f"Return value (sdf) type:    {type(rv_try[1])}")
    print(f"Return value (sdf) schema:")
    rv_try[1].printSchema()
    print(f"Return value (sdf):")
    rv_try[1].show(truncate = False, n = 5)
    print(f"Return value (sdf_count) type: {type(rv_try[2])}")
    print(f"Return value (sdf_count):      {rv_try[2]}")
    print("\n--------------------\n")
    print(f"\n# Execution (except) - Start #")
    dict_parameter = {
        "technolog": "Excel",
        "frequenc": "Daily",
        "server_nam": "Reference",
        "database_nam": "Data",
        "schema_nam": "Rapport_sur les coûts de dist. des addendas",
        "table_nam": "Rapport"
    }
    time_start = dt.datetime.now()
    rv_except = fn_add_metadata_column(sdf, dict_parameter)
    print(f"# Execution (except) - End (duration: {(dt.datetime.now() - time_start)}) #\n")
    print(f"Return value:               {rv_except}")
    print(f"Return value (status) type: {type(rv_except[0])}")
    print(f"Return value (status):      {rv_except[0]}")
    print(f"Return value (ex) type:     {type(rv_except[1])}")
    print(f"Return value (ex):          {rv_except[1]}")
    print(f"\n----- Test fn_add_metadata_column() - End -----\n")

# Operation

## Execute "Silver"

In [ ]:
def fn_execute_silver():
    print("Execute Silver")

In [ ]:
#mssparkutils.notebook.exit("OK")